In [1]:
import meshio
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from torch_geometric.transforms import FaceToEdge
import matplotlib.pyplot as plt

In [2]:
class TetraToEdge(object):
    r"""Converts mesh tetras :obj:`[4, num_tetras]` to edge indices
    :obj:`[2, num_edges]`.
    Args:
        remove_tetras (bool, optional): If set to :obj:`False`, the tetra tensor
            will not be removed.
    """

    def __init__(self, remove_tetras=True):
        self.remove_tetras = remove_tetras

    def __call__(self, data):
        if data.tetra is not None:
            tetra = data.tetra
            edge_index = torch.cat([tetra[:2], tetra[1:3, :], tetra[-2:], tetra[::2], tetra[::3], tetra[1::2]], dim=1)
            edge_index = to_undirected(edge_index, num_nodes=data.num_nodes)

            data.edge_index = edge_index
            if self.remove_tetras:
                data.tetra = None

        return data

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)

In [16]:
def from_meshio(mesh, mesh_type='2D'):
    r"""Converts a :.msh file to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        mesh (meshio.read): A :obj:`meshio` mesh.
    """

    if meshio is None:
        raise ImportError('Package `meshio` could not be found.')

    pos = torch.from_numpy(mesh.points).to(torch.float)
    if mesh_type == '3D':
        tetra = torch.from_numpy(mesh.cells_dict['tetra']).to(torch.long).t().contiguous()
        face = torch.from_numpy(mesh.cells_dict['triangle']).to(torch.long).contiguous()
        return Data(pos=pos, tetra=tetra, face=face)
    elif mesh_type == '2D':
        face = torch.from_numpy(mesh.cells_dict['triangle']).to(torch.long).t().contiguous()
        return Data(pos=pos, face=face)

In [17]:
mesh_type = '3D'
filename = '../meshes/sphere_coarse.msh'

mesh = meshio.read(filename)

data = from_meshio(mesh, mesh_type=mesh_type)
if mesh_type == '2D':
    data = FaceToEdge(remove_faces=False)(data)
else:
    data = TetraToEdge(remove_tetras=False)(data)

data.tetra tiene información de los índices de cada tetraedro. [vértices (4), tetraedros]

In [19]:
data

Data(edge_index=[2, 11746], face=[1126, 3], pos=[1001, 3], tetra=[4, 4310])

In [60]:
face = data.face[457]

In [61]:
face

tensor([153, 184, 183])

In [67]:
(data.tetra == face[0]).any(0)

tensor([False, False, False,  ..., False, False, False])

In [62]:
torch.where((data.tetra == face[0]).any(0) & (data.tetra == face[1]).any(0) & (data.tetra == face[2]).any(0))

(tensor([3288]),)

In [63]:
torch.where((data.tetra == face[0]).any(0) & (data.tetra == face[1]).any(0))

(tensor([3288, 4063]),)

In [64]:
data.tetra[:,35], data.tetra[:,990], data.tetra[:,1092]

(tensor([113, 706, 114, 756]),
 tensor([406, 113, 114, 756]),
 tensor([ 32, 114, 113, 706]))

In [15]:
for face in data.face:
    

tensor([[ 19,   0,  28,  ..., 465, 481, 513],
        [ 28,  42, 536,  ..., 551, 514, 544],
        [  0,  19,   0,  ..., 478, 504, 519]])

In [11]:
idx_tetra = 1

In [12]:
data.tetra[:,idx_tetra]

tensor([568, 688, 642, 694])

In [13]:
face_vertex = data.tetra[:,idx_tetra][:3]

In [24]:
import numpy as np

In [32]:
base = np.array([[1,3,4],
                [2,6,3]])
search = np.array([2,3])

In [33]:
search.shape

(2,)

In [34]:
(base == search[:,None]).any(axis=1)

array([False,  True])